In [1]:
import pandas as pd

In [2]:
scripts = pd.read_csv('data/processed_scripts.csv')
meta = pd.read_csv('data/metadata.csv')

In [3]:
scripts.head()

,Season,Episode,Title,Speaker,Dialog,S-E
0,4,10,The Girl From Poughkeepsie,Ross,hey,04-10
1,4,10,The Girl From Poughkeepsie,Ross,im sorry im late did i miss anything,04-10
2,4,10,The Girl From Poughkeepsie,Phoebe,joey stuffing 15 oreos in his mouth,04-10
3,4,10,The Girl From Poughkeepsie,Ross,15 your personal best,04-10
4,4,10,The Girl From Poughkeepsie,Phoebe,where were you,04-10


In [4]:
meta.head()

,S-E,Viewers,Stars
0,01-01,21.5,8.3
1,01-02,20.2,8.1
2,01-03,19.5,8.2
3,01-04,19.7,8.1
4,01-05,18.6,8.5
